In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

## Import and clean the dataset

In [2]:
#Import the dataset for Math class
X_train = pd.read_csv("student-mat.csv", sep=";")
X_train.shape

(395, 33)

In [3]:
#a function to onehot encode the dataset
def OneHot(dataset):
    #One-hot encode
    #seperate columns that have numerical data and categorical data. 
    num_col = dataset._get_numeric_data().columns
    cat_col = list(set(dataset.columns) - set(num_col))
    
    dataset = pd.get_dummies(dataset, columns =cat_col, drop_first = True)
       
    return dataset

In [4]:
X_train = OneHot(X_train)
Y_train = X_train[["G3"]].values
X_train = X_train.drop("G3", axis=1).values

In [5]:
#instantiate the scaler object and scale the training dataset
scaler = MinMaxScaler(feature_range=(0,1))
X_train = scaler.fit_transform(X_train)
Y_train = scaler.fit_transform(Y_train)

C:\Users\TimeL\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


## Define the NN model

In [6]:
#parameters used to define the tensorflow NN model
learning_rate = 0.001
epochs = 100   #training iterations
dropout = 0.8  #percentage of neurons to keep from dropout

inputNodes = X_train.shape[1]   #number of input features
outputNodes = Y_train.shape[1]  #output labels being predicted

layer1_size = 50
layer2_size = 100
layer3_size = 50

In [7]:
#Input layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, inputNodes))

# Hidden Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable("weights1", shape=[inputNodes, layer1_size], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer1_size], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)
    layer_1_output = tf.nn.dropout(layer_1_output, keep_prob = dropout)

# Hidden Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable("weights2", shape=[layer1_size, layer2_size], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer2_size], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)
    layer_2_output = tf.nn.dropout(layer_2_output, keep_prob = dropout)

# Hidden Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable("weights3", shape=[layer2_size, layer3_size], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer3_size], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)
    layer_3_output = tf.nn.dropout(layer_3_output, keep_prob = dropout)

# Output Layer
with tf.variable_scope('output'):
    weights = tf.get_variable("weights4", shape=[layer3_size, outputNodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[outputNodes], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

In [8]:
# Define the cost and train functions
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

# Define the optimizer function that will be run to optimize the neural network
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


## Train the model

In [9]:
# Run the session to train
with tf.Session() as session:
    # Initialize all the variables
    session.run(tf.global_variables_initializer())
    
    for epoch in range (epochs):
        # run the optimizer and display cost
        session.run(optimizer, feed_dict={X: X_train, Y: Y_train})
        
        # Every 5 training steps, log our progress
        if epoch % 5 == 0:
            training_cost = session.run(cost, feed_dict={X: X_train, Y:Y_train})
            print(epoch, training_cost)

    # Training is now complete!
    print("Training is complete!")
    session.close()

0 0.3135015
5 0.1219297
10 0.11123484
15 0.08223857
20 0.071259975
25 0.06525059
30 0.0597432
35 0.06154408
40 0.058933325
45 0.054409157
50 0.055719994
55 0.048371684
60 0.044485513
65 0.04812342
70 0.048279718
75 0.04398458
80 0.047589213
85 0.043088358
90 0.041406985
95 0.0386762
Training is complete!
